# Returns

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create transaction portfolio

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query, we create a Transaction Portfolio in LUSID
-- ============================================================

-- Defining scope and code variables
@@portfolioScope =

select 'IBOR';

@@portfolioCode =

select 'uk-equity';

-- Define the portfolio data
@create_portfolio =

select 'Transaction' as PortfolioType, @@portfolioScope as PortfolioScope, @@portfolioCode as
   PortfolioCode, @@portfolioCode as DisplayName, '' as Description, #2000-01-01# as Created, ''
   as SubHoldingKeys, 'GBP' as BaseCurrency;

-- Upload the portfolio into LUSID
@response_create_portfolio =

select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

select *
from @response_create_portfolio;


#### Step 2: Upload portfolio returns

In [ ]:
%%luminesce

-- =======================================================
-- Description:
-- In this query, we upload performance data into LUSID 
-- =======================================================

-- Defining scope and code variables
@@portfolioScope =

select 'IBOR';

@@portfolioCode =

select 'uk-equity';

-- Load performance data from .csv
@performance_data_csv =

use Drive.Csv
--file=/luminesce-examples/performance_data.csv
enduse;

-- Define the Returns data
@performance_data = 

select  @@portfolioScope as PortfolioScope, 
        @@portfolioCode as PortfolioCode, 
        'Production' as ReturnScope, 
        'Performance' as ReturnCode,
        'Daily' as Period, 
        return_date as EffectiveAt, 
        mv as OpeningMarketValue, 
        returns as RateOfReturn
from @performance_data_csv;

-- Upload returns into LUSID
@response_performance_data =

select *
from Lusid.Portfolio.Return.Writer
where ToWrite = @performance_data;

select *
from @response_performance_data;



#### Step 3: Calculate aggregate returns

In [ ]:
%%luminesce

-- =======================================================
-- Description:
-- In this query, we use the portfolio created in step 1
-- and return values from step 2 to aggreate our returns
-- with various metrics
-- =======================================================

-- Defining scope and code variables
@@portfolioScope =

select 'IBOR';

@@portfolioCode =

select 'uk-equity';

-- Define values to be matched with throughout the portfolios
@lookup_table =

select @@portfolioScope as PortfolioScope,
       @@portfolioCode as PortfolioCode,
       'Production' as ReturnScope,
       'Performance' as ReturnCode;

-- Define performance metrics
@perfMetrics =  select '1D' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, '1 Day' as Alias
                union all
                select 'INC' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Inception' as Alias
                union all
                select 'WTD' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Week-to-Date' as Alias
                union all
                select 'MTD' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Month-to-Date' as Alias
                union all
                select 'QTD' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Quarter-to-Date' as Alias
                union all
                select 'YTD' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Year-to-Date' as Alias
                union all
                select '1M-ROLLING' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, '1 Month Rolling' as Alias
                union all
                select '3M-ROLLING' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, '3 Month Rolling' as Alias
                union all
                select '1Y-ROLLING' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, '1 Year Rolling' as Alias
                union all
                select 'SINCE(2019-01-01)' as WindowMetric, false as AllowPartial, false as WithFee, false as Annualised, 'Since 01-01-2019' as Alias
                union all
                select 'INC' as WindowMetric, false as AllowPartial, false as WithFee, true as Annualised, 'Annualised since Inception' as Alias;

-- Aggregate returns from portfolio
select *
from Lusid.Portfolio.AggregatedReturn
where ToLookUp = @lookup_table
and PerformanceReturnMetrics = @perfMetrics;
